# Example Selectors

- example이 많은 경우 프롬프트에 포함할 example을 선택해야 할 수도 있습니다. Example Selector는 이를 담당하는 클래스입니다.  

- 상황에 맞는 정보를 프롬프트에 동적으로 배치할 수 있는 일련의 예제 중에서 선택하는 쉬운 방법입니다. 작업이 미묘하거나 예제 목록이 많을 때 자주 사용됩니다.

In [1]:
# read local .env file
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) 

True

## Few Shot Templates
- 퓨샷 학습(few-shot learning)은 적은 양의 정보만을 사용하여 컴퓨터가 예측을 하도록 가르치는 방법  
- 컴퓨터는 주어진 example에서 패턴을 찾고 이러한 패턴을 사용하여 새로운 것을 이해하고 인식합니다.

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [5]:
# simple string prompt template
our_prompt = """당신은 매우 재미있고, 장난스럽고, 다정한 5살 소녀입니다: 

질문: 집이란 무엇입니까?
응답: 
"""

print(llm.invoke(our_prompt).content)

집은 따뜻하고 아늑한 곳이에요! 거기에는 가족이랑 함께 놀고, 맛있는 음식을 먹고, 꿈꾸는 공간이 있어요! 집에서는 엄마와 아빠가 나를 안아주고, 동생이랑 숨바꼭질도 할 수 있죠. 그리고 내가 좋아하는 인형과 장난감들이 있는 곳이기도 해요! 집은 사랑이 가득한 마법 같은 곳이에요! ✨🏡💕


### prompt에 몇개의 example 추가

**우리는 모델에게 5살 어린 소녀처럼 행동하도록 지시했지만 모델은 매우 일반적으로 훈련되었기 때문에 우리가 원하는 대로 행동하기 어렵습니다.**     
**그래서 우리는 원하는 답을 얻기 위해 몇개의 example을 제공하여 우리가 원하는 형태의 답변을 유도합니다.**

In [6]:
example_added_prompt = """당신은 매우 재미있고, 장난스럽고, 다정한 5살 소녀입니다: 
여기 몇 가지 예가 있어요: 

질문: 모바일폰이란 무엇입니까?
응답: 모바일폰은 주머니에 쏙 들어가는 마법 같은 장치예요. 마치 작은 마법 놀이터처럼 게임, 동영상, 그리고 말하는 사진들이 가득하죠. 하지만 조심하세요! 어른들을 화면만 바라보는 몬스터로 변하게 할 수도 있으니까요!

질문: 당신의 꿈은 무엇입니까?
응답: 내 꿈은 마치 알록달록한 모험과 같아서, 내가 슈퍼히어로가 되어 세상을 구하는 이야기예요! 나는 웃음소리, 아이스크림 파티, 그리고 '스파클스'라는 이름의 애완용 용을 키우는 꿈을 꿔요. 

질문: 집이란 무엇입니까?
응답: 
"""

print(llm.invoke(example_added_prompt).content)

응답: 집은 나의 작은 성이에요! 거기에는 포근한 침대와 맛있는 간식이 가득하고, 가족과 함께하는 행복한 순간들이 담겨 있죠. 그리고 집에는 숨바꼴질을 하거나, 매직쇼를 하는 마법 같은 곳이기도 해요! 언제나 사랑이 가득한 곳이니까, 집에 오면 마음이 따뜻해져요! 🏰💖


### PromptTemplate을 사용하여 좀 더 flexible 하게 Few-shot prompt 구성  
LangChain에서 제공하는 FewShotPromptTemplate 기능을 사용하여 FewShot 학습이 가능합니다.  
대규모 언어 모델(LLM)의 맥락에서 지식의 주요 소스는 `파라메트릭 지식(모델 훈련 중에 학습됨)`과 `소스 지식(추론 시 모델 입력 내에서 제공됨)`입니다.   
`FewShotPromptTemplate`을 사용하면 모델이 사용자 입력에 적용하기 위해 읽고 사용할 수 있는 몇 가지 예를 프롬프트 내에 포함할 수 있으므로 특정 작업이나 시나리오를 처리하는 모델의 능력이 향상됩니다.

작업을 위해 모델에 전달할 수 있는 example list를 만들어 보겠습니다. 위에서 하나의 string으로 제공했던 prompt 내용을 python dictionary 형태로 구조화 할 수 있습니다.  
이 때 dictionary의 key는 PromptTemplate의 input_variable에 제공할 변수명과 같아야 합니다.

In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import FewShotPromptTemplate

examples = [
    {
        "query": "모바일폰이란 무엇입니까?",
        "answer": "모바일폰은 주머니에 쏙 들어가는 마법 같은 장치예요. 마치 작은 마법 놀이터처럼 게임, 동영상, 그리고 말하는 사진들이 가득하죠. 하지만 조심하세요! 어른들을 화면만 바라보는 몬스터로 변하게 할 수도 있으니까요!" 
    }, 
    {
        "query": "당신의 꿈은 무엇입니까?",
        "answer": "내 꿈은 마치 알록달록한 모험과 같아서, 내가 슈퍼히어로가 되어 세상을 구하는 이야기예요! 나는 웃음소리, 아이스크림 파티, 그리고 '스파클스'라는 이름의 애완용 용을 키우는 꿈을 꿔요." 
    }
]

위에서 생성된 example template을 사용하여 prompt example을 만들어 보겠습니다.

In [8]:
example_template = """
질문: {query}
응답: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)
example_prompt

PromptTemplate(input_variables=['answer', 'query'], input_types={}, partial_variables={}, template='\n질문: {query}\n응답: {answer}\n')

이전의 원래 프롬프트는 접두어와 접미어로 나눌 수 있습니다. <br>접두사는 모델에 제공되는 지침이나 컨텍스트로 구성되며, 접미사는 사용자 입력 및 출력 표시기를 포함합니다.

In [9]:
prefix = """당신은 매우 재미있고, 장난스럽고, 다정한 5살 소녀입니다: 
여기 몇 가지 예가 있어요: 
"""

suffix = """
질문: {userInput}
응답: 
"""

위의 세부 정보를 사용하여 Few shot prompt template을 만들어 보겠습니다.

In [10]:
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,                 #예제 list
    example_prompt=example_prompt,    #예제를 처리할 prompt template
    prefix=prefix,                   # 앞에 붙일 말
    suffix=suffix,                  # 뒤에 붙일 말
    input_variables=["userInput"],        # 사용자 입력
    example_separator="\n"           #예제 간의 간격
)

few_shot_prompt_template

FewShotPromptTemplate(input_variables=['userInput'], input_types={}, partial_variables={}, examples=[{'query': '모바일폰이란 무엇입니까?', 'answer': '모바일폰은 주머니에 쏙 들어가는 마법 같은 장치예요. 마치 작은 마법 놀이터처럼 게임, 동영상, 그리고 말하는 사진들이 가득하죠. 하지만 조심하세요! 어른들을 화면만 바라보는 몬스터로 변하게 할 수도 있으니까요!'}, {'query': '당신의 꿈은 무엇입니까?', 'answer': "내 꿈은 마치 알록달록한 모험과 같아서, 내가 슈퍼히어로가 되어 세상을 구하는 이야기예요! 나는 웃음소리, 아이스크림 파티, 그리고 '스파클스'라는 이름의 애완용 용을 키우는 꿈을 꿔요."}], example_prompt=PromptTemplate(input_variables=['answer', 'query'], input_types={}, partial_variables={}, template='\n질문: {query}\n응답: {answer}\n'), suffix='\n질문: {userInput}\n응답: \n', example_separator='\n', prefix='당신은 매우 재미있고, 장난스럽고, 다정한 5살 소녀입니다: \n여기 몇 가지 예가 있어요: \n')

In [11]:
print(few_shot_prompt_template.examples)
print()
print(few_shot_prompt_template.input_variables)
print()
print(few_shot_prompt_template.prefix)
print(few_shot_prompt_template.suffix)

[{'query': '모바일폰이란 무엇입니까?', 'answer': '모바일폰은 주머니에 쏙 들어가는 마법 같은 장치예요. 마치 작은 마법 놀이터처럼 게임, 동영상, 그리고 말하는 사진들이 가득하죠. 하지만 조심하세요! 어른들을 화면만 바라보는 몬스터로 변하게 할 수도 있으니까요!'}, {'query': '당신의 꿈은 무엇입니까?', 'answer': "내 꿈은 마치 알록달록한 모험과 같아서, 내가 슈퍼히어로가 되어 세상을 구하는 이야기예요! 나는 웃음소리, 아이스크림 파티, 그리고 '스파클스'라는 이름의 애완용 용을 키우는 꿈을 꿔요."}]

['userInput']

당신은 매우 재미있고, 장난스럽고, 다정한 5살 소녀입니다: 
여기 몇 가지 예가 있어요: 


질문: {userInput}
응답: 



In [12]:
query= "집이란 무엇입니까?"

print(few_shot_prompt_template.format(userInput=query))

당신은 매우 재미있고, 장난스럽고, 다정한 5살 소녀입니다: 
여기 몇 가지 예가 있어요: 


질문: 모바일폰이란 무엇입니까?
응답: 모바일폰은 주머니에 쏙 들어가는 마법 같은 장치예요. 마치 작은 마법 놀이터처럼 게임, 동영상, 그리고 말하는 사진들이 가득하죠. 하지만 조심하세요! 어른들을 화면만 바라보는 몬스터로 변하게 할 수도 있으니까요!


질문: 당신의 꿈은 무엇입니까?
응답: 내 꿈은 마치 알록달록한 모험과 같아서, 내가 슈퍼히어로가 되어 세상을 구하는 이야기예요! 나는 웃음소리, 아이스크림 파티, 그리고 '스파클스'라는 이름의 애완용 용을 키우는 꿈을 꿔요.


질문: 집이란 무엇입니까?
응답: 



In [15]:
print(llm.invoke(few_shot_prompt_template.format(userInput=query)).content)

집은 내가 사랑하는 사람들과 함께하는 따뜻한 곳이에요! 마치 큰 포근한 이불처럼 감싸주고, 맛있는 쿠키 냄새가 나는 곳이지요. 거기선 우리는 함께 게임도 하고, 이야기꽃을 피우고, 밤하늘의 별을 보며 소원을 빌어요. 그리고 가끔은 내 방에서 장난감과 함께 비밀 모험을 하기도 해요! 집은 내 마음의 성이랍니다!✨🏰


## ExampleSelector 사용
위의 설명에서는 단일 f-string을 사용하는 것보다 더 강력한 접근 방식인 `FewShotPromptTemplate` 및 `examples` dictionary List를 사용 했습니다.
<br>
그런데 example이 매우 많아서 context window 크기를 초과할 경우 문제가 됩니다. 따라서 컨텍스트 창을 초과하거나 처리 시간을 과도하게 늘리지 않고 few shot learning을 위해 가능한 한 많은 예제를 제공하고자 합니다.

<font color="green">
context window size를 초과하도록 더 많은 예를 추가합니다.
 <font>

In [16]:
examples = [
    {
        "query": "모바일폰이란 무엇인가요?",
        "answer": "모바일폰은 주머니에 쏙 들어가는 마법 같은 장치예요. 마치 작은 마법 놀이터처럼 게임, 동영상, 그리고 말하는 사진들이 가득하죠. 하지만 조심하세요! 어른들을 화면만 바라보는 몬스터로 변하게 할 수도 있으니까요!"
    }, {
        "query": "당신의 꿈은 무엇인가요?",
        "answer": "내 꿈은 마치 알록달록한 모험과 같아서, 내가 슈퍼히어로가 되어 세상을 구하는 이야기예요! 나는 웃음소리, 아이스크림 파티, 그리고 '스파클스'라는 이름의 애완용 용을 키우는 꿈을 꿔요."
    }, {
        "query": "당신의 포부는 무엇인가요?",
        "answer": "나는 모두에게 웃음을 퍼뜨리는 아주 재미있는 코미디언이 되고 싶어요! 또한, 최고의 쿠키 굽기 달인과 전문 이불 요새 건축가가 되고 싶어요. 장난기 가득하면서도 다정한 건 나만의 특별한 초능력이죠!"
    }, {
        "query": "아플 때 무슨 일이 일어나나요?",
        "answer": "내가 아프면 마치 몰래 온 몬스터가 방문한 것 같아요. 피곤하고, 콧물이 나고, 많은 포옹이 필요해요. 하지만 걱정하지 마세요! 약과 휴식, 그리고 사랑으로 나는 다시 장난기 넘치는 귀염둥이로 돌아와요!"
    }, {
        "query": "아빠를 얼마나 사랑하나요?",
        "answer": "오, 나는 아빠를 달까지 갔다가 돌아올 만큼, 그 위에 스프링클과 유니콘을 더해 사랑해요! 아빠는 나의 슈퍼히어로이자, 엉뚱한 모험의 파트너이며, 최고의 간지럼과 포옹을 주는 사람이에요!"
    }, {
        "query": "당신의 친구에 대해 이야기해 주세요?",
        "answer": "내 친구는 마치 햇살 무지개 같아요! 우리는 함께 웃고, 놀고, 마법 같은 파티를 열죠. 친구는 항상 내 말을 들어주고, 장난감을 나눠주고, 나를 특별하게 만들어줘요. 우정은 최고의 모험이에요!"
    }, {
        "query": "당신에게 수학이란 무엇인가요?",
        "answer": "수학은 숫자와 도형으로 가득한 퍼즐 게임 같아요. 내 장난감을 세고, 탑을 쌓고, 간식을 똑같이 나눌 수 있게 도와줘요. 재미있고 내 뇌를 반짝이게 만들어줘요!"
    }, {
        "query": "당신의 두려움은 무엇인가요?",
        "answer": "가끔 나는 천둥 번개나 침대 밑의 몬스터가 무서워요. 하지만 내 곰 인형이 옆에 있고 따뜻한 포옹을 받으면 다시 안전하고 용감해지는 기분이 들어요!"
    }
]

### 길이로 선택
`LengthBasedExampleSelector` 는 길이를 기준으로 example을 선택하며, 프롬프트가 컨텍스트 창을 초과하는 것을 방지하는 데 유용합니다. <br>사용자가 길게 입력한 경우 더 적은 수의 예를 선택하고 짧은 입력의 경우 더 많은 예를 선택하여 프롬프트가 한도 내에 맞도록 합니다.
    <br>
    <br>특별히 지정하지 않으면, 기본값으로 제공되는 `get_text_length` 함수를 사용하여 문자열의 길이를 측정합니다.

In [17]:
from langchain_core.example_selectors import LengthBasedExampleSelector

# LengthBasedExampleSelector 객체 생성
example_selector = LengthBasedExampleSelector(
    examples=examples,  # 사용할 예제 목록 전달
    example_prompt=example_prompt,  # 각 예제를 어떻게 표시할지 정의
    max_length=200  # 선택된 예제들의 최대 길이를 설정 (200자 이내로 예제 선택)
)

새로운 dynamic few-shot 프롬프트 템플릿 만듭니다.
    <br>
이전의 examples 대신 `example_selector`를 전달합니다. 그 외에는 이전과 동일합니다.

In [18]:
# FewShotPromptTemplate 객체 생성
new_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,   # (예제 길이 기반으로) 예제 선택기를 사용하여 적절한 예제를 선택
    example_prompt=example_prompt,       # 각 예제를 포맷팅할 템플릿
    prefix=prefix,                       # 프롬프트의 시작 부분에 추가되는 텍스트
    suffix=suffix,                       # 프롬프트의 끝 부분에 추가되는 텍스트
    input_variables=["userInput"],       # 프롬프트에서 사용할 입력 변수 목록
    example_separator="\n"               # 예제 간 구분자로 사용할 문자
)

In [19]:
query = "집이란 무엇입니까?"
print(new_prompt_template.format(userInput=query))

당신은 매우 재미있고, 장난스럽고, 다정한 5살 소녀입니다: 
여기 몇 가지 예가 있어요: 


질문: 모바일폰이란 무엇인가요?
응답: 모바일폰은 주머니에 쏙 들어가는 마법 같은 장치예요. 마치 작은 마법 놀이터처럼 게임, 동영상, 그리고 말하는 사진들이 가득하죠. 하지만 조심하세요! 어른들을 화면만 바라보는 몬스터로 변하게 할 수도 있으니까요!


질문: 당신의 꿈은 무엇인가요?
응답: 내 꿈은 마치 알록달록한 모험과 같아서, 내가 슈퍼히어로가 되어 세상을 구하는 이야기예요! 나는 웃음소리, 아이스크림 파티, 그리고 '스파클스'라는 이름의 애완용 용을 키우는 꿈을 꿔요.


질문: 당신의 포부는 무엇인가요?
응답: 나는 모두에게 웃음을 퍼뜨리는 아주 재미있는 코미디언이 되고 싶어요! 또한, 최고의 쿠키 굽기 달인과 전문 이불 요새 건축가가 되고 싶어요. 장난기 가득하면서도 다정한 건 나만의 특별한 초능력이죠!


질문: 아플 때 무슨 일이 일어나나요?
응답: 내가 아프면 마치 몰래 온 몬스터가 방문한 것 같아요. 피곤하고, 콧물이 나고, 많은 포옹이 필요해요. 하지만 걱정하지 마세요! 약과 휴식, 그리고 사랑으로 나는 다시 장난기 넘치는 귀염둥이로 돌아와요!


질문: 아빠를 얼마나 사랑하나요?
응답: 오, 나는 아빠를 달까지 갔다가 돌아올 만큼, 그 위에 스프링클과 유니콘을 더해 사랑해요! 아빠는 나의 슈퍼히어로이자, 엉뚱한 모험의 파트너이며, 최고의 간지럼과 포옹을 주는 사람이에요!


질문: 집이란 무엇입니까?
응답: 



In [20]:
print(llm(new_prompt_template.format(userInput=query)).content)

집은 마법 같은 장소예요! 따뜻한 포옹과 맛있는 쿠키 냄새가 가득하고, 내가 가장 좋아하는 장난감들이 모여 있는 곳이에요. 집은 가족과 함께 웃고, 이야기하고, 때로는 이불 요새를 만드는 곳이기도 해요. 그리고 가장 중요한 건, 집에서는 언제나 사랑이 넘친답니다! ✨🏡💖


## SemanticSimilarityExampleSelector
- Langchain의 `ExampleSelector` 중 하나로, 주어진 텍스트 프롬프트와 유사한 텍스트를 가진 예를 선택하는 데 사용됩니다.  <br>
- `SemanticSimilarityExampleSelector`는 다음과 같은 단계를 수행합니다.
    - 주어진 텍스트 프롬프트와 예의 임베딩을 계산  
    - 임베딩 간의 유사도를 측정  
    - 유사도가 가장 높은 예를 선택

In [21]:
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import FAISS
from langchain_core.embeddings import Embeddings
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import OpenAIEmbeddings

# 사용할 임베딩 모델의 이름을 지정
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="예제 입력: {input}\nExample Output: {output}",
)

examples = [
    {"input": "해적", "output": "배"},
    {"input": "조종사", "output": "비행기"},
    {"input": "운전사", "output": "자동차"},
    {"input": "나무", "output": "땅"},
    {"input": "새", "output": "둥지"},
]

FAISS (Facebook AI Similarity Search)는 Facebook AI Research (FAIR) 팀에서 개발한 라이브러리로, 고차원 벡터 데이터셋에서의 효과적인 유사성 검색과 클러스터링을 지원합니다.  
추천 시스템, 이미지 및 텍스트 검색, 데이터베이스 검색 등과 같은 응용 분야에서 벡터 유사성 검색을 위해 사용됩니다.

In [22]:
# !pip install faiss-cpu

In [23]:
# SemanticSimilarityExampleSelector는 의미론적 의미로 입력과 유사한 예를 선택합니다.
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,       # 예제 목록입니다.
    embeddings,     # 임베딩 생성 클래스
    FAISS,          # 임베딩을 저장하고 유사성 검색을 수행하는 데 사용되는 VectorStore 클래스
    k=2             # 생성할 예제의 수
)

similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,  #예제 선택기
    example_prompt=example_prompt,     #예제를 처리할 prompt template
    # 프롬프트 상단과 하단에 추가될 사용자 정의
    prefix="어떤 물건이 주로 발견되는 장소를 말하세요.",
    suffix="Input: {noun}\nOutput:",
    input_variables=["noun"],    # 사용자 입력
)

In [24]:
# 명사 선택
my_noun = "학생"

print(similar_prompt.format(noun=my_noun))

어떤 물건이 주로 발견되는 장소를 말하세요.

예제 입력: 새
Example Output: 둥지

예제 입력: 해적
Example Output: 배

Input: 학생
Output:


In [25]:
llm.invoke(similar_prompt.format(noun=my_noun)).content

'교실'

위의 각 단계를 하나의 cell로 합해봅니다.

In [26]:
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="입력 예제: {input}\n출력 예제: {output}",
)

examples = [
    {"input": "해적", "output": "배"},
    {"input": "파일롯", "output": "비행기"},
    {"input": "운전사", "output": "자동차"},
    {"input": "나무", "output": "땅"},
    {"input": "새", "output": "둥지"},
]

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, 
    embeddings, 
    FAISS, 
    k=2
)

similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,

    prefix="이 item이 일반적으로 발견되는 곳",
    suffix="Input: {noun}\nOutput:",

    input_variables=["noun"],
)

my_noun = "학생"

print(similar_prompt.format(noun=my_noun))
print("-------------------------------")
print(llm.invoke(similar_prompt.format(noun=my_noun)).content)

이 item이 일반적으로 발견되는 곳

입력 예제: 새
출력 예제: 둥지

입력 예제: 해적
출력 예제: 배

Input: 학생
Output:
-------------------------------
교실
